<a href="https://colab.research.google.com/github/Dgab8898/ML_project_predicting_heart_disease/blob/master/Project_movie_recommender_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [41]:
# Intall Tensorflow
!pip install -q tensorflow-gpu==2.0.0-rc0 
# For the current version: 
!pip install --upgrade tensorflow

# For a specific version:
!pip install tensorflow==1.2
# For the latest nightly build:
!pip install tf-nightly
try:
  %tensorflow_version 2.x
except Exception:
  pass

import tensorflow as tf
print(tf.__version__)


ERROR: tf-nightly 2.5.0.dev20210109 has requirement tb-nightly~=2.5.0.a, but you'll have tb-nightly 1.15.0a20190806 which is incompatible.
ERROR: tf-nightly 2.5.0.dev20210109 has requirement tf-estimator-nightly~=2.5.0.dev, but you'll have tf-estimator-nightly 1.14.0.dev2019080601 which is incompatible.
     |████████████████████████████████| 394.7MB 41kB/s 
     |████████████████████████████████| 2.9MB 47.1MB/s 
     |████████████████████████████████| 3.8MB 45.8MB/s 
ERROR: tf-nightly 2.5.0.dev20210109 has requirement gast==0.4.0, but you'll have gast 0.3.3 which is incompatible.
ERROR: tf-nightly 2.5.0.dev20210109 has requirement grpcio~=1.34.0, but you'll have grpcio 1.32.0 which is incompatible.
ERROR: tf-nightly 2.5.0.dev20210109 has requirement h5py~=3.1.0, but you'll have h5py 2.10.0 which is incompatible.
ERROR: tf-nightly 2.5.0.dev20210109 has requirement tb-nightly~=2.5.0.a, but you'll have tb-nightly 1.15.0a20190806 which is incompatible.
ERROR: tf-nightly 2.5.0.dev20210109 

  Using cached https://files.pythonhosted.org/packages/5e/55/7995cc1e9e60fa37ea90e6777d832e75026fde5c6109215d892aaff2e9b7/tensorflow-1.2.0-cp36-cp36m-manylinux1_x86_64.whl
Processing /root/.cache/pip/wheels/b6/52/17/f0af18e3e0ec6fa60b361ffed15b4c3468f6f3bcdb87fbe079/Markdown-2.2.0-cp36-none-any.whl
ERROR: tensorboard 2.4.0 has requirement markdown>=2.6.8, but you'll have markdown 2.2.0 which is incompatible.
ERROR: tb-nightly 1.15.0a20190806 has requirement markdown>=2.6.8, but you'll have markdown 2.2.0 which is incompatible.
  Found existing installation: Markdown 3.3.3
    Uninstalling Markdown-3.3.3:
      Successfully uninstalled Markdown-3.3.3
  Found existing installation: tensorflow 2.4.0
    Uninstalling tensorflow-2.4.0:
      Successfully uninstalled tensorflow-2.4.0


  Using cached https://files.pythonhosted.org/packages/3e/d4/178c6faf76c958b1ad48e928c3e139853c5be5681ba2eaf3ac66434a6d67/tf_estimator_nightly-2.5.0.dev2021010901-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/e6/36/25f9ad05a602ebd61e079cb79f206c039bf6d93460b1f01e88395cd8504a/tb_nightly-2.5.0a20210108-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/91/ab/8a7b37278fb59f3688af01cd069a5a4d2f3383eea2a2f78ddea4c7be047a/grpcio-1.34.0-cp36-cp36m-manylinux2014_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/b6/48/583c032b79ae5b3daa02225a675aeb673e58d2cb698e78510feceb11958c/gast-0.4.0-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/70/7a/e53e500335afb6b1aade11227cdf107fca54106a1dca5c9d13242a043f3b/h5py-3.1.0-cp36-cp36m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/ac/ef/24a91ca96efa0d7802dffb83ccc7a3c677027bea19ec3c9ee80be740408e/Markdown-3.3.3-py3-none-any.whl
ERROR: te

2.4.0


In [81]:
# More import

!apt-get -qq install -y libarchive-dev && pip install -U libarchive
import libarchive
!apt-get -qq install -y graphviz && pip install pydot
import pydot
!apt-get -qq install python-cartopy python3-cartopy
import cartopy

from tensorflow.keras.layers import Input, Dense, Embedding, Flatten, \
  Concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import SGD, Adam
from sklearn.utils import shuffle




import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

 # the dataset form: https://
 # we interested in 2o million review
!wget -nc http://files.grouplens.org/datasets/movielens/ml-20m.zip

Requirement already up-to-date: libarchive in /usr/local/lib/python3.6/dist-packages (0.4.7)
File ‘ml-20m.zip’ already there; not retrieving.



In [30]:
!unzip -n ml-20m.zip

Archive:  ml-20m.zip


In [31]:
!ls

ml-20m	ml-20m.zip  sample_data


In [ ]:
ml-20m ml-20m.zp sample_data

In [32]:
df = pd.read_csv('ml-20m/ratings.csv')
df.head()

,userId,movieId,rating,timestamp
0,1,2,3.5,1112486027
1,1,29,3.5,1112484676
2,1,32,3.5,1112484819
3,1,47,3.5,1112484727
4,1,50,3.5,1112484580


In [33]:
# We cannot trust UserId and MoviesId to be 0...N-1
# lets set our own id


df.userId = pd.Categorical(df.userId)
df['new_user_id'] = df.userId.cat.codes

df.movieId = pd.Categorical(df.movieId)
df['new_movie_id'] = df.movieId.cat.codes

In [34]:
user_ids = df['new_user_id'].values
movie_ids = df['new_movie_id'].values
ratings = df['rating'].values

In [65]:
# The number of user_id, and movie_id
N = len(set(user_ids))
M = len(set(movie_ids))

# Set embbeding dimension
K = 10

In [70]:

# neural network

# user input
u = Input(shape=(1,))

# movie input
m = Input(shape=(1,))

# user embedding
u_emb = Embedding(N, K)(u)
# movie embedding
m_emb = Embedding(M, K)(m)

# Flatten user and movie embedding
u_emb = Flatten()(u_emb)
m_emb = Flatten()(m_emb)

# Concatenate into feature vectors
x = Concatenate()([u_emb, m_emb])


#
x = Dense(1204, activation='relu')(x)
# x = Dense(400, activation='relu')(x)
# x = Dense(400, activation='relu')(x)
x = Dense(1)(x)

In [74]:
# Build the model and compile them
import keras
from keras.models import load_model
from keras.models import Sequential

model = Model(inputs=[u, m], outputs=x)
model.compile(
    loss='mse', 
    optimizer=SGD(lr=0.08, momentum=0.9),
    )

In [77]:
# Split the dataset
user_ids, movie_ids, ratings = shuffle(user_ids, movie_ids, ratings)
Ntrain = int(0.8 * len(ratings))
train_user = user_ids[:Ntrain]
train_movie = movie_ids[:Ntrain]
train_ratings = ratings[:Ntrain]

test_user = user_ids[Ntrain:]
test_movie = movie_ids[Ntrain:]
test_ratings = ratings[Ntrain:]

# Centre the data
avg_rating = train_ratings.mean()
train_ratings = train_ratings - avg_rating
test_ratings = test_ratings - avg_rating

In [87]:

# r = model.fit(
#     x=[train_user, train_movie], 
#     y=train_ratings,
#     epochs=25,
#     batch_size=1024,
#     verbose=2,)
# validation_data=([test_user, test_movie], test_ratings)


In [ ]:
# Plot the losses
plt.plot(r.history['loss'], label="train loss")
plt.plot(r.history['val_loss'], label="val loss")
plt.legend()
plt.show()

In [ ]:
np.sqrt(0.2569)